In [3]:
import pickle
from collections import Counter
import pdb
from IPython.core.debugger import set_trace
import time
import util
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
data = pickle.load(open('data_processed.p', 'rb'))

In [5]:
data[0]

['buffalo',
 '?',
 '66',
 'is',
 'a',
 'very',
 'rarely',
 'known',
 'movie',
 'that',
 'stars',
 'vincent',
 'gallo',
 'and',
 'christina',
 'ricci',
 'gallo',
 'plays',
 'a',
 'very',
 'troubled',
 'man',
 'who',
 'was',
 'sent',
 'to',
 'jail',
 'for',
 'gambling',
 'once',
 'out',
 'of',
 'jail',
 'he',
 'must',
 'visit',
 'his',
 'parents',
 'who',
 'he',
 'told',
 'he',
 'was',
 'married',
 'the',
 'truth',
 'is',
 'he',
 "isn't",
 'married',
 'to',
 'try',
 'to',
 'impress',
 'them',
 'he',
 'kidnaps',
 'a',
 'girl',
 'christina',
 'ricci',
 'from',
 'a',
 'tap',
 'dancing',
 'class',
 'to',
 'act',
 'as',
 'his',
 'wife',
 'the',
 'film',
 'is',
 'very',
 'cheaply',
 'made',
 'and',
 'it',
 'shows',
 'it',
 'throughout',
 'a',
 'lot',
 'of',
 'the',
 'movie',
 'but',
 'you',
 "don't",
 'need',
 'money',
 'to',
 'make',
 'a',
 'good',
 'film',
 'buffalo',
 '?',
 '66',
 "doesn't",
 'always',
 'stay',
 'with',
 'the',
 'realistic',
 'concept',
 'and',
 'sometimes',
 'goes',
 'thro

# problem 4,5

In [6]:
def convert_to_sparse_BOW(review_list):
    bow_of_reviews = []
    for reviews in review_list:
        bow_of_reviews.append(Counter(reviews))
    return bow_of_reviews

In [7]:
bow = convert_to_sparse_BOW(data)

In [8]:
bow[0]

Counter({'buffalo': 4,
         '?': 4,
         '66': 4,
         'is': 11,
         'a': 12,
         'very': 12,
         'rarely': 1,
         'known': 1,
         'movie': 4,
         'that': 5,
         'stars': 1,
         'vincent': 1,
         'gallo': 4,
         'and': 14,
         'christina': 2,
         'ricci': 4,
         'plays': 3,
         'troubled': 1,
         'man': 3,
         'who': 3,
         'was': 5,
         'sent': 1,
         'to': 13,
         'jail': 2,
         'for': 1,
         'gambling': 1,
         'once': 1,
         'out': 2,
         'of': 3,
         'he': 7,
         'must': 1,
         'visit': 1,
         'his': 4,
         'parents': 3,
         'told': 1,
         'married': 2,
         'the': 20,
         'truth': 1,
         "isn't": 3,
         'try': 1,
         'impress': 1,
         'them': 1,
         'kidnaps': 1,
         'girl': 1,
         'from': 2,
         'tap': 1,
         'dancing': 1,
         'class': 1,
         'act'

In [9]:
train = bow[:int(len(bow)*0.75)]
val = bow[int(len(bow)*0.75):]

# problem 6-4. pegasos 알고리즘 구현

In [10]:
class svm_pegasos():
    def __init__(self, lambda_reg = 1):
        if lambda_reg < 0:
            raise ValueError('Regularization penalty should be at least 0.')
        self.lambda_reg = lambda_reg
    
    def fit(self, X, epoch = 1):
        self.w_ = dict()
        t = 0
#         set_trace()
        for _ in range(epoch):
            for j in range(len(X)):
                t += 1
                step_size = 1/(t*self.lambda_reg)
                w_dot_x = util.dotProduct(self.w_,X[j])
                y = 1 if 1 in X[j] else -1


                if y*w_dot_x < 1:
                    util.increment(self.w_, -step_size*self.lambda_reg, self.w_)
                    util.increment(self.w_, step_size*y, X[j])
                

                else:
                    util.increment(self.w_, -step_size*self.lambda_reg, self.w_)
                   
                
        return self.w_
        
    

In [11]:
svm = svm_pegasos(0.1)

#### 학습 시간 측정

In [12]:
start = time.time()
w = svm.fit(train, 2)
end = time.time()
print(end - start)

16.39494276046753


In [13]:
w['66']

0.02000000000000004

# problem 6-5~8. pegasos 알고리즘을 더 빠르게 실행되도록 구현

In [14]:
class svm_pegasos_sW():
    def __init__(self, lambda_reg = 1):
        if lambda_reg < 0:
            raise ValueError('Regularization penalty should be at least 0.')
        self.lambda_reg = lambda_reg
    
    def fit(self, X, epoch = 1):
        W = dict()
        t = 0
        s = 1
#         set_trace()
        for _ in range(epoch):
            for j in range(len(X)):
                t += 1
                step_size = 1/(t*self.lambda_reg)                
                y = 1 if 1 in X[j] else -1            
                w_dot_x = util.dotProduct(W,X[j])*s
                
                if y*w_dot_x < 1:
                    s = (1 - step_size*self.lambda_reg)*s
                    if s == 0:
                        s = 1
                        W = dict()         
                    util.increment(W, step_size*y/s, X[j])
                    
                else:
                    s = (1 - step_size*self.lambda_reg)*s
              
        util.increment(W,s-1,W)
        self.w_ = W
        return self.w_
    
    def zero_one_loss(self, X):
        try:
            getattr(self, "w_")
        except AttributeError:
            raise RuntimeError("You must train classifer before predicting data!")
            
        loss = 0
        for j in range(len(X)):
            y = 1 if 1 in X[j] else -1
            w_dot_x = util.dotProduct(self.w_, X[j]) ## score 계산
            
            if y*w_dot_x < 0:
                loss += 1
        
        return loss/len(X)
    
    def find_error_example(self, X):
        try:
            getattr(self, "w_")
        except AttributeError:
            raise RuntimeError("You must train classifer before predicting data!")
        
        error_exapmles = [] ## 예측을 제대로 못한 예제
        correct_examples = [] ## 예측을 제대로한 예제
        for j in range(len(X)):
            y = 1 if 1 in X[j] else -1
            w_dot_x = util.dotProduct(self.w_, X[j])
            
            if y*w_dot_x < 0:
                error_exapmles.append((X[j], w_dot_x))
            
            elif y*w_dot_x > 0:
                correct_examples.append((X[j], w_dot_x))
                
        
        return error_exapmles, correct_examples

In [15]:
svm_sW = svm_pegasos_sW(0.1)

#### 학습 시간 측정. 그냥 한 것 보다 훨씬 빠르다. 

In [16]:
start = time.time()
w_sW = svm_sW.fit(train, 2)
end = time.time()
print(end - start)

0.29326534271240234


#### 기존의 것과 결과가 같다.

In [17]:
w['buffalo']

0.02333333333333335

In [18]:
w_sW['buffalo']

0.023333333333326323

#### zero-one loss 측정

In [19]:
loss = svm_sW.zero_one_loss(val)
loss

0.208

#### lambda값 설정
  - loss가 0이 나오다니 데이터가 lineary separable한건가?

In [20]:
lambda_values = [1e-7, 1e-5, 1e-3, 1e-1, 1, 10, 1e3, 1e5]
for lambda_reg in lambda_values:
    svm = svm_pegasos_sW(lambda_reg)
    svm.fit(train,2)
    loss = svm.zero_one_loss(val)
    print("loss {} at lambda: {}".format(loss, lambda_reg))

loss 0.368 at lambda: 1e-07
loss 0.0 at lambda: 1e-05
loss 0.368 at lambda: 0.001
loss 0.208 at lambda: 0.1
loss 0.276 at lambda: 1
loss 0.312 at lambda: 10
loss 0.496 at lambda: 1000.0
loss 0.496 at lambda: 100000.0


In [21]:
lambda_values = [1e-5, 2e-5, 3e-5, 4e-5, 5e-5, 6e-5, 7e-5, 8e-5, 9e-5, 10e-5,]
for lambda_reg in lambda_values:
    svm = svm_pegasos_sW(lambda_reg)
    svm.fit(train,2)
    loss = svm.zero_one_loss(val)
    print("loss {} at lambda: {}".format(loss, lambda_reg))

loss 0.0 at lambda: 1e-05
loss 0.0 at lambda: 2e-05
loss 0.368 at lambda: 3e-05
loss 0.0 at lambda: 4e-05
loss 0.368 at lambda: 5e-05
loss 0.368 at lambda: 6e-05
loss 0.368 at lambda: 7e-05
loss 0.0 at lambda: 8e-05
loss 0.368 at lambda: 9e-05
loss 0.368 at lambda: 0.0001


In [22]:
lambda_values = [1e-5, 2e-5, 3e-5, 4e-5, 5e-5, 6e-5, 7e-5, 8e-5, 9e-5, 10e-5,]
for lambda_reg in lambda_values:
    svm = svm_pegasos_sW(lambda_reg)
    svm.fit(train,15)
    loss = svm.zero_one_loss(val)
    print("loss {} at lambda: {}".format(loss, lambda_reg))

loss 0.0 at lambda: 1e-05
loss 0.0 at lambda: 2e-05
loss 0.156 at lambda: 3e-05
loss 0.0 at lambda: 4e-05
loss 0.156 at lambda: 5e-05
loss 0.156 at lambda: 6e-05
loss 0.156 at lambda: 7e-05
loss 0.0 at lambda: 8e-05
loss 0.156 at lambda: 9e-05
loss 0.156 at lambda: 0.0001


 - train에 대해서도 loss가 0인걸 보니 lineary separable한 데이터인게 맞는 것 같다.

In [23]:
svm = svm_pegasos_sW(1e-5)

svm.fit(train, 2)
loss = svm.zero_one_loss(train)
loss

0.0

In [24]:
svm = svm_pegasos_sW(8e-5)

svm.fit(train, 2)
loss = svm.zero_one_loss(train)
loss

0.0

In [25]:
svm = svm_pegasos_sW(3e-5)

svm.fit(train, 2)
loss = svm.zero_one_loss(train)
loss

0.35533333333333333

# problem 7. error analysis

In [26]:
svm = svm_pegasos_sW(0.1)
w = svm.fit(train, 2)

In [27]:
errors, correct = svm.find_error_example(val)

In [28]:
len(errors)

104

In [29]:
for e in errors: ## 정답을 맞추지 못한 데이터들에 대한 모델의 score값 확인.
    print(e[1])

1.2500000000012061
5.1066666666664045
2.919999999999284
3.4300000000002804
3.1333333333329474
-2.3866666666672565
10.74999999999869
-2.3366666666666163
-0.24666666666571757
1.9433333333335359
6.189999999999658
7.556666666666075
1.763333333332886
-7.029999999997042
0.5900000000011278
0.7066666666667754
0.06666666666726484
-2.923333333332584
2.9833333333330323
-0.9899999999995597
2.5199999999997544
-4.539999999999296
5.86999999999982
0.01333333333424308
-1.0866666666653675
0.11333333333381812
1.5200000000005254
2.610000000000859
-5.2499999999986695
6.276666666665983
1.6433333333329383
4.4799999999991655
4.579999999999986
4.980000000000992
-5.0466666666638496
-3.0999999999980936
-5.689999999998118
0.29666666666730457
3.063333333334265
7.179999999999506
-3.643333333332297
2.3166666666672935
5.933333333332435
3.310000000000615
4.153333333334997
-0.06666666666622767
0.6333333333333453
0.6166666666669229
1.4100000000014958
3.719999999999553
0.5366666666659778
8.249999999998773
0.0433333333349

In [30]:
def calculate_wx(w, error):
    wx_list = []
    for feature in error:
        if feature in w:
            wx_list.append((w[feature]*error[feature], feature, w[feature], error[feature], abs(w[feature]*error[feature]) ))
    wx_list.sort(key = lambda x : x[4], reverse = True)
    return wx_list
    

In [49]:
len(errors[8][0])

310

In [53]:
sample_neg_1 = calculate_wx(w, errors[0][0])
errors[0][1] ## 첫번째 에러의 스코어값. 이건 예측을 제대로 못한 데이터이므로 스코어가 양수인 것은 실제 y는 -1인 것이다.

1.2500000000012061

In [54]:
sample_neg_1

[(6.269999999998618, 'and', 0.32999999999992724, 19, 6.269999999998618),
 (5.733333333333007, 'the', 0.13333333333332575, 43, 5.733333333333007),
 (2.12333333333288, 'in', 0.16333333333329847, 13, 2.12333333333288),
 (-1.909999999999627, 'have', -0.6366666666665424, 3, 1.909999999999627),
 (1.7099999999995248, 'is', 0.08999999999997499, 19, 1.7099999999995248),
 (-1.6166666666663332, 'on', -0.32333333333326664, 5, 1.6166666666663332),
 (-1.1099999999996726, "there's", -0.36999999999989086, 3, 1.1099999999996726),
 (-1.0999999999996817, 'too', -0.36666666666656056, 3, 1.0999999999996817),
 (-1.0799999999996999, 'to', -0.08999999999997499, 12, 1.0799999999996999),
 (-1.0599999999999454, 'only', -0.5299999999999727, 2, 1.0599999999999454),
 (-1.0499999999997272, 'like', -0.34999999999990905, 3, 1.0499999999997272),
 (-1.0266666666666424, 'an', -0.2566666666666606, 4, 1.0266666666666424),
 (0.933333333333394, 'also', 0.466666666666697, 2, 0.933333333333394),
 (-0.853333333333012, 'even', -

In [55]:
sample_neg_2 = calculate_wx(w, errors[1][0])
errors[1][1] ## 두번째 에러의 스코어 값

5.1066666666664045

In [56]:
sample_neg_2

[(5.93999999999869, 'and', 0.32999999999992724, 18, 5.93999999999869),
 (5.199999999999704, 'the', 0.13333333333332575, 39, 5.199999999999704),
 (-1.919999999999618, 'movie', -0.31999999999993634, 6, 1.919999999999618),
 (1.6333333333329847, 'in', 0.16333333333329847, 10, 1.6333333333329847),
 (1.400000000000091, 'he', 0.2333333333333485, 6, 1.400000000000091),
 (-1.2933333333330665, 'on', -0.32333333333326664, 4, 1.2933333333330665),
 (-1.2733333333330847, 'have', -0.6366666666665424, 2, 1.2733333333330847),
 (1.2599999999996498, 'is', 0.08999999999997499, 14, 1.2599999999996498),
 (-0.8099999999997749, 'to', -0.08999999999997499, 9, 0.8099999999997749),
 (-0.7966666666666242, 'bad', -0.7966666666666242, 1, 0.7966666666666242),
 (-0.75, 'any', -0.25, 3, 0.75),
 (-0.7466666666664423, 'about', -0.18666666666661058, 4, 0.7466666666664423),
 (-0.6999999999998181, 'like', -0.34999999999990905, 2, 0.6999999999998181),
 (-0.6333333333332121, 'no', -0.31666666666660603, 2, 0.6333333333332121)

In [57]:
sample_neg_3 = calculate_wx(w, errors[2][0])
errors[2][1]  ## 3번째 에러의 스코어 값

2.919999999999284

In [58]:
sample_neg_3

[(4.949999999998909, 'and', 0.32999999999992724, 15, 4.949999999998909),
 (3.733333333333121, 'the', 0.13333333333332575, 28, 3.733333333333121),
 (1.6333333333329847, 'in', 0.16333333333329847, 10, 1.6333333333329847),
 (-1.0266666666666424, 'an', -0.2566666666666606, 4, 1.0266666666666424),
 (-0.7966666666666242, 'bad', -0.7966666666666242, 1, 0.7966666666666242),
 (-0.7733333333330847, 'off', -0.19333333333327118, 4, 0.7733333333330847),
 (0.7199999999997999, 'is', 0.08999999999997499, 8, 0.7199999999997999),
 (-0.6766666666664491, 'with', -0.09666666666663559, 7, 0.6766666666664491),
 (-0.6466666666665333, 'on', -0.32333333333326664, 2, 0.6466666666665333),
 (-0.6299999999998249, 'to', -0.08999999999997499, 7, 0.6299999999998249),
 (-0.5799999999998136, 'this', -0.19333333333327118, 3, 0.5799999999998136),
 (-0.5733333333332666, 'at', -0.2866666666666333, 2, 0.5733333333332666),
 (-0.5299999999999727, 'only', -0.5299999999999727, 1, 0.5299999999999727),
 (-0.4999999999998863, 'her'

In [61]:
sample_neg_4 = calculate_wx(w, errors[3][0])
errors[3][1]

3.4300000000002804

In [62]:
sample_neg_4

[(5.93999999999869, 'and', 0.32999999999992724, 18, 5.93999999999869),
 (5.733333333333007, 'the', 0.13333333333332575, 43, 5.733333333333007),
 (1.4699999999996862, 'in', 0.16333333333329847, 9, 1.4699999999996862),
 (-1.279999999999518, 'even', -0.426666666666506, 3, 1.279999999999518),
 (1.1699999999996749, 'is', 0.08999999999997499, 13, 1.1699999999996749),
 (1.1666666666667425, 'he', 0.2333333333333485, 5, 1.1666666666667425),
 (-0.9899999999997249, 'to', -0.08999999999997499, 11, 0.9899999999997249),
 (-0.959999999999809, 'movie', -0.31999999999993634, 3, 0.959999999999809),
 (-0.9466666666667152, 'be', -0.2366666666666788, 4, 0.9466666666667152),
 (0.8800000000001091, 'what', 0.22000000000002728, 4, 0.8800000000001091),
 (0.786666666666406, 'you', 0.393333333333203, 2, 0.786666666666406),
 (-0.7333333333331211, 'too', -0.36666666666656056, 2, 0.7333333333331211),
 (-0.6999999999998181, 'like', -0.34999999999990905, 2, 0.6999999999998181),
 (-0.6466666666665333, 'on', -0.32333333

In [59]:
sample_pos_1 = calculate_wx(w, errors[5][0])
errors[5][1]

-2.3866666666672565

In [60]:
sample_pos_1

[(10.266666666666083, 'the', 0.13333333333332575, 77, 10.266666666666083),
 (9.833333333330074, 'you', 0.393333333333203, 25, 9.833333333330074),
 (8.249999999998181, 'and', 0.32999999999992724, 25, 8.249999999998181),
 (-7.039999999998599, 'movie', -0.31999999999993634, 22, 7.039999999998599),
 (-5.093333333332339, 'have', -0.6366666666665424, 8, 5.093333333332339),
 (-3.1799999999998363, 'only', -0.5299999999999727, 6, 3.1799999999998363),
 (-3.093333333332339, 'this', -0.19333333333327118, 16, 3.093333333332339),
 (3.0599999999991496, 'is', 0.08999999999997499, 34, 3.0599999999991496),
 (-2.9099999999993997, 'on', -0.32333333333326664, 9, 2.9099999999993997),
 (-2.8799999999991996, 'to', -0.08999999999997499, 32, 2.8799999999991996),
 (2.506666666666206, 'will', 0.31333333333327573, 8, 2.506666666666206),
 (-2.339999999999918, '?', -0.2599999999999909, 9, 2.339999999999918),
 (-2.2166666666662422, 'no', -0.31666666666660603, 7, 2.2166666666662422),
 (1.9800000000002456, 'what', 0.22